<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [2]:
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 323 (delta 35), reused 21 (delta 20), pack-reused 268 (from 2)
Receiving objects: 100% (323/323), 94.95 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [3]:
%cd MT-Preparation

/content/MT-Preparation


In [4]:
!pip3 install --user -r requirements.txt

In [5]:
%cd ..

/content


In [6]:
!ls

MT-Preparation	sample_data


In [ ]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de


In [ ]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

2025-10-28 14:04:38.833889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761660278.853572    1551 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761660278.859542    1551 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761660278.874791    1551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761660278.874840    1551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761660278.874844    1551 computation_placer.cc:177] computation placer alr

In [ ]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**

I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of 11,654,416 sentences. I took the first 100k sentences to create the corpus, which will be splitted in the following notebooks into train, val and test sets.

As the corpus (txt.files) was too big to load into Colab, I uploaded the them into my Google Drive to open it here and take only some sentences.

The created files are:

*   MultiUN.en-ru.en
*   MultiUN.en-ru.ru
*   MultiUN.en-ru.en-filtered.en
*   MultiUN.en-ru.ru-filtered.ru
*   MultiUN.en-ru.en
*   MultiUN.en-ru.ru


Link: https://opus.nlpl.eu/MultiUN/en&ru/v1/MultiUN


## Corpus creation

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
file_en = "/content/drive/MyDrive/Colab Notebooks/MultiUN.en-ru.en"
file_ru = "/content/drive/MyDrive/Colab Notebooks/MultiUN.en-ru.ru"

In [35]:
def extract_and_save_first_n_lines(file_path, output_path, n_lines):

    extracted_lines = 0
    with open(file_path, 'r', encoding='utf-8') as fin, \
         open(output_path, 'w', encoding='utf-8') as fout:
        for i, line in enumerate(fin):
            if i >= n_lines:
                break
            fout.write(line)
            extracted_lines += 1

    print(f"Extracted and saved {extracted_lines:,} lines from '{file_path}' to '{output_path}'")
    return output_path

In [36]:
output_file_ru = "/content/MultiUN.en-ru.ru"
output_file_en = "/content/MultiUN.en-ru.en"

n_lines = 100000

extract_and_save_first_n_lines(file_ru, output_file_ru, n_lines=n_lines)
extract_and_save_first_n_lines(file_en, output_file_en, n_lines=n_lines)

Extracted and saved 100,000 lines from '/content/drive/MyDrive/Colab Notebooks/MultiUN.en-ru.ru' to '/content/MultiUN.en-ru.ru'
Extracted and saved 100,000 lines from '/content/drive/MyDrive/Colab Notebooks/MultiUN.en-ru.en' to '/content/MultiUN.en-ru.en'


'/content/MultiUN.en-ru.en'

In [37]:
for i in range(7):
  print(train_ru[i])
  print(train_en[i])

Совместное совещание ЕЭК/Евростата/ОЭСР по вопросам управления статистическими информационными системами (УСИС)

Joint ECE/Eurostat/OECD Meeting on the Management of Statistical Information Systems (MSIS)

(Братислава, Словакия, 18-20 апреля 2005 года)

(Bratislava, Slovakia, 18-20 April 2005)

Тема i): Практика управления ИТ в статистических управлениях

Topic (i): IT governance in statistical offices

УПРАВЛЕНИЕ ИНФОРМАЦИОННОЙ ТЕХНОЛОГИЕЙ В УНС -

INFORMATION TECHNOLOGY GOVERNANCE @ ONS - DO WE NEED IT GOVERNANCE IN A STATISTICAL OFFICE?

Специальный документ

Invited Paper

Представлен Управлением национальной статистики Соединенного Королевства

Submitted by the Office for National Statistics, United Kingdom

Резюме

Summary



In [38]:
print(len(train_ru))
print(type(train_ru))
print(len(train_en))
print(type(train_en))

100000
<class 'list'>
100000
<class 'list'>


## Corpus preparation

In [30]:
!python3 MT-Preparation/filtering/filter.py MultiUN.en-ru.en MultiUN.en-ru.ru en ru

Dataframe shape (rows, columns): (100000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 99999
--- Duplicates Deleted			--> Rows: 93728
--- Source-Copied Rows Deleted		--> Rows: 91965
--- Too Long Source/Target Deleted	--> Rows: 86996
--- Too Short Source/Target Deleted	--> Rows: 86910
--- HTML Removed			--> Rows: 86910
--- Rows will remain true-cased		--> Rows: 86910
--- Rows with Empty Cells Deleted	--> Rows: 86910
--- Rows Shuffled			--> Rows: 86910
--- Source Saved: MultiUN.en-ru.en-filtered.en
--- Target Saved: MultiUN.en-ru.ru-filtered.ru


In [43]:
with open('/content/MultiUN.en-ru.en-filtered.en', 'r') as f:
    text = f.read()
print(text[:100])
print(len(text))

Women thus won a broad-based agenda for promoting and protecting their human rights worldwide, while
14662673


In [44]:
with open('/content/MultiUN.en-ru.ru-filtered.ru', 'r') as f:
    text = f.read()
print(text[:100])
print(len(text))

Женщины добились принятия широкой программы поощрения и защиты их прав во всем мире, наряду с утверж
16026119


In [42]:
!python3 MT-Preparation/filtering/semantic_filter.py MultiUN.en-ru.en-filtered.en  MultiUN.en-ru.ru-filtered.ru  en ru 5000 0.45

2025-10-31 23:29:41.895278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761953381.926572    6663 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761953381.936350    6663 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761953381.959695    6663 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761953381.959729    6663 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761953381.959736    6663 computation_placer.cc:177] computation placer alr

KeyboardInterrupt: 

In [ ]:
with open('/content/MultiUN.en-ru.en-filtered.en.semantic.en', 'r') as f:
    text = f.read()
print(text[:100])
print(len(text))

In [ ]:
with open('/content/MultiUN.en-ru.ru-filtered.ru.semantic.ru', 'r') as f:
    text = f.read()
print(text[:100])
print(len(text))